In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
import torch.nn.functional as F
from torchvision import transforms, datasets, models

In [2]:
# define global variables
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
print(f"using {DEVICE}")
EPOCHS = 50
BATCH_SIZE = 20

using cuda


In [3]:
# data preprocessing
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
trainset = datasets.ImageFolder(root='./data/Candlesticks/coin_Bitcoin/20days/ohlc/train',
transform=transform)
testset = datasets.ImageFolder(root='./data/Candlesticks/coin_Bitcoin/20days/ohlc/test',
transform=transform)

train_loader = data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    #num_workers = 4
)
test_loader = data.DataLoader(
    dataset = testset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    #num_workers = 4
)

In [4]:
class ConvNet(nn.Module):
    def __init__(self, n_classes=2):
        super(ConvNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(10),
            nn.Dropout2d(p=0.2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(20),
            nn.Dropout2d(p=0.2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(20*61*61, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(64, n_classes),
        )
    
    def forward(self, x):
        x = self.model(x)
        return x

In [5]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        #print(inputs, labels)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

In [6]:
def evaluate(model, train_loader, test_loader):
    model.eval()
    train_loss = 0.
    test_loss = 0.
    train_correct = 0.
    test_correct = 0.
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            train_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            train_correct += pred.eq(target.view_as(pred)).sum().item()

        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            test_correct += pred.eq(target.view_as(pred)).sum().item()
    train_loss /= len(train_loader.dataset)
    train_acc = 100.*train_correct / len(train_loader.dataset)
    test_loss /= len(test_loader.dataset)
    test_acc = 100.*test_correct / len(test_loader.dataset)
    return train_loss, train_acc, test_loss, test_acc

In [7]:
#model = ConvNet(n_classes=2)
#model = models.resnet18()
model = models.vgg16_bn()
print(model)
model = model.to(DEVICE)
optimizer = optim.Adadelta(model.parameters(), lr=0.1, rho=0.9, eps=1e-06, weight_decay=0.0005)
#optimizer = optim.Adam(model.parameters(), lr=0.1)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.05)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [8]:
history = dict()
history["train_loss"] = list()
history["train_acc"] = list()
history["test_loss"] = list()
history["test_acc"] = list()

for epoch in range(EPOCHS): 
    train(model, train_loader, optimizer)
    scheduler.step()
    train_loss, train_acc, test_loss, test_acc = evaluate(model, train_loader, test_loader)
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['test_loss'].append(test_loss)
    history['test_acc'].append(test_acc)
        
    print(f"[{epoch}] Train Loss: {round(train_loss, 4)}, Test Loss: {round(test_loss, 4)}, Accuracy: {round(test_acc, 2)}")
print('Finished Training')

[0] Train Loss: 1.4202, Test Loss: 1.4673, Accuracy: 43.17
[1] Train Loss: 0.7322, Test Loss: 0.7101, Accuracy: 56.83
[2] Train Loss: 0.9957, Test Loss: 0.9874, Accuracy: 43.72
[3] Train Loss: 0.6963, Test Loss: 0.7011, Accuracy: 50.0
[4] Train Loss: 0.7453, Test Loss: 0.7257, Accuracy: 56.83
[5] Train Loss: 0.7077, Test Loss: 0.6994, Accuracy: 56.28
[6] Train Loss: 0.6905, Test Loss: 0.6866, Accuracy: 56.83
[7] Train Loss: 1.0046, Test Loss: 0.9709, Accuracy: 56.83
[8] Train Loss: 0.6858, Test Loss: 0.6848, Accuracy: 58.2
[9] Train Loss: 0.6893, Test Loss: 0.6945, Accuracy: 57.1
[10] Train Loss: 0.6793, Test Loss: 0.6846, Accuracy: 57.65
[11] Train Loss: 0.6799, Test Loss: 0.6873, Accuracy: 56.56
[12] Train Loss: 0.6785, Test Loss: 0.6849, Accuracy: 57.38
[13] Train Loss: 0.6768, Test Loss: 0.6865, Accuracy: 56.56
[14] Train Loss: 0.6793, Test Loss: 0.69, Accuracy: 53.83
[15] Train Loss: 0.6786, Test Loss: 0.6902, Accuracy: 54.1
[16] Train Loss: 0.6764, Test Loss: 0.6847, Accuracy: 57

KeyboardInterrupt: 

In [ ]:
epochs = np.arange(1, EPOCHS+1)
train_loss = np.array(history['train_loss'])
test_loss = np.array(history['test_loss'])
train_acc = np.array(history['train_acc'])
test_acc = np.array(history['test_acc'])

plt.figure(figsize=(12, 12))
plt.subplot(2, 1, 1)
plt.plot(epochs, train_loss, "b", label="train loss")
plt.plot(epochs, test_loss, "r", label="test loss")
plt.legend(loc='best')

plt.subplot(2, 1, 2)
plt.plot(epochs, train_acc, "b", label="train acc")
plt.plot(epochs, test_acc, "r", label="test acc")
plt.legend(loc='best')